In [4]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
import numpy as np
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot

In [ ]:
# Gan de imagens 640x640 de 3 canais de cancer de pele
# imagens /datasets/train/images
path = '/datasets/train/images'

# Discriminador
def define_discriminator(in_shape=(640,640,3)):
    model = Sequential()
    # normal
    model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# Gerador
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 20x20 image
    n_nodes = 128 * 20 * 20
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((20, 20, 128)))
    # upsample to 40x40
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 80x80
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 160x160
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 320x320
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # output layer
    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
    return model 

# GAN
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

# Carregar imagens
def load_real_samples():
    # pegar imagens
    X = np.load(path)
    # converter de int para float
    X = X.astype('float32')
    # escalar de -1 a 1
    X = (X - 127.5) / 12
    return X

# Treina com todas as imagens
def generate_real_samples(dataset, n_samples):
    # escolher imagens aleatorias
    ix = randint(0, dataset.shape[0], n_samples)
    # selecionar imagens
    X = dataset[ix]
    # gerar classe
    y = ones((n_samples, 1))
    return X, y

# Gerar pontos no espaço latente
def generate_latent_points(latent_dim, n_samples):
    # gerar pontos no espaço latente
    x_input = randn(latent_dim * n_samples)
    # redimensionar para o modelo
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# Usar o gerador para criar imagens falsas
def generate_fake_samples(g_model, latent_dim, n_samples):
    # gerar pontos no espaço latente
    x_input = generate_latent_points(latent_dim, n_samples)
    # prever
    X = g_model.predict(x_input)
    # criar classe
    y = zeros((n_samples, 1))
    return X, y


